# Blackjack Simulator

### What it does
   <ul>
        <li>Simulate n blackjack games between two players</li>
        <li>Uses six decks to simulate casino games</li>
        <li>Plots what the player does when comparing their cards to the known dealer dealers</li>
   </ul> 


## Import necessary packages

In [30]:
import random
import pandas as pd
import tensorflow as tf
from abc import ABC, abstractmethod
from math import *

## Creating player and dealer classes

In [31]:
class Player(ABC):
    
    def __init__(self):
        self.__current_cards = []
        self.__value = 0
        self.wins = 0
        self.games_played = 0
        self.__aces = 0
    
    def __str__(self):
        to_print = "My current cards are {0}, with a value of {1}. I currently have {2} wins.".format(" ".join(map(str,self.__current_cards)),
                                                                                                       self.__value,
                                                                                                       self.wins)
        return to_print
    
    def __repr__(self):
        if self.games_played > 0:
            return str(round(self.wins/self.games_played,2))
        return str(0)
        
    def returnStats(self):
        return self.wins/self.games_played
    
    def seeCards(self):
        return self.__current_cards
    
    def getValue(self):
        return self.__value
    
    def resetCards(self):
        self.__current_cards = []
        self.__value = 0
        self.__aces = 0
    
    def verifyAmount(self):
        ## Verify the amount of the player and see if they win or lose
        
        if self.__value > 21:
            return 'L'
        elif self.__value == 21:
            return 'W'
        else:
            return 'C'
        
    def giveCard(self, card):
        ## Give a player a card and update the value, their hand and then verify the amount that they have
        
        self.__current_cards.append(card)
        self.__updateValue(card)
        self.verifyAmount()
    
    def __updateValue(self, card):
        ## Updates the player's hand values depending on what card they get
        
        if(card in ['Q','J','K']):
            card = 10
        if card == 1 and self.__value + 11 < 21:
            # If the value of the card is a one, then it can either be 11 or just 1
            self.__value += 11
            self.__aces += 1
        
        elif self.__aces > 0 and self.__value + card > 21:
            #If we have a lot of aces and their value if they were 11 exceeds 21, then convert them into ones
            self.__value -= 11
            self.__aces -= 1
            self.__value += card + 1
        
        else:
            self.__value += card
    
    def updatePlayer(self, winOrLose):
        if winOrLose == True:
            #Player won
            self.wins += 1
        self.games_played += 1
   
    @abstractmethod
    def makeDecision(self, oppositeValue):
        pass
        
        
class Dealer(Player):
    
    def __init__(self):
        Player.__init__(self)
        self.__deckOfCards = {}
        self.__firstTurn = 0
        for x in range(1, 11):
            self.__deckOfCards[x] = 24
        self.__deckOfCards['Q'] = 24
        self.__deckOfCards['J'] = 24
        self.__deckOfCards['K'] = 24
    
    def playerHit(self, Player):
        self.__firstTurn += 1
        while(True):
            card = random.choice(list(self.__deckOfCards))

            if(self.__deckOfCards[card] > 0):
                self.__deckOfCards[card] -= 1
                break
        Player.giveCard(card)
    
    def seeCards(self):
        if self.__firstTurn <= 2:
            return Player.seeCards(self)[0]
        else:
            return Player.seeCards(self)
    
    def makeDecision(self, oppositeValue):
        if random.randint(0,2) == 1:
            return True # Indicates hit and get another card
        else:
            return False
    
    def resetCards(self):
        Player.resetCards(self)
        self.__firstTurn = True
        for x in range(1, 11):
            self.__deckOfCards[x] = 24
        self.__deckOfCards['Q'] = 24
        self.__deckOfCards['J'] = 24
        self.__deckOfCards['K'] = 24

new_dealer = Dealer()
new_dealer.playerHit(new_dealer)
print(new_dealer)


My current cards are 2, with a value of 2. I currently have 0 wins.


## Create a game class to handle the games automatically

In [32]:
class Game:
    
    def __init__(self, playerModel, dealerModel):
        self.__turn = True # False is dealer turn, True is player turn
        self.numberOfGames = 0
        self.__seed = random.seed()
        self.player = playerModel
        self.dealer = dealerModel
        self.__state = 'F'
    
    def printStats(self):
        print('Dealer: %f %% Win Rate' % (self.dealer.wins / self.dealer.games_played * 100))
        print('Player: %f %% Win Rate' % (self.player.wins / self.player.games_played * 100))
    
    def resetPlayers(self):
        self.player.resetCards()
        self.dealer.resetCards()
        self.__state = 'F'
        
    def SimulateGames(self,**kwargs):
        for i in range(1, kwargs['n']+1):

            self.dealer.playerHit(self.player)
            self.dealer.playerHit(self.player)
            self.dealer.playerHit(self.dealer)
            self.dealer.playerHit(self.dealer)
            
            playerDecision = self.player.makeDecision(self.player.getValue()) # Returns True if player decides to hit
            while (playerDecision == True and self.__state == 'C') or self.__state == 'F':
                self.dealer.playerHit(self.player)
                self.__state = self.__VerifyStates()
                playerDecision = self.player.makeDecision(self.player.getValue())

            dealerDecision = self.dealer.makeDecision(self.dealer.getValue()) # Returns False otherwise
            while dealerDecision == True and self.__state == 'C':
                self.dealer.playerHit(self.dealer)
                self.__state = self.__VerifyStates()
                dealerDecision = self.dealer.makeDecision(self.dealer.getValue())
            
            
            self.__state = self.__VerifyStates()
            
            if(self.__state == 'D' or (self.__state == 'C' and self.player.getValue() < self.dealer.getValue())):
                # Dealer Wins
                self.dealer.updatePlayer(True)
                self.player.updatePlayer(False)
            elif(self.__state == 'P' or (self.__state == 'C' and self.player.getValue() > self.dealer.getValue())):
                # Player Wins
                self.dealer.updatePlayer(False)
                self.player.updatePlayer(True)
            else:
                # Tie
                self.dealer.updatePlayer(False)
                self.player.updatePlayer(False)
            
            self.resetPlayers()
        
    def __VerifyStates(self):
        # Determine if we should continue the game or not.
        # If not, return a code to indicate
        player_status = self.player.verifyAmount()
        dealer_status = self.dealer.verifyAmount()
        if player_status == 'L':
            return 'D' # Dealer Wins
        elif dealer_status == 'L':
            return 'P' # Player Wins
        elif player_status == 'W' and dealer_status == 'W':
            return 'T' # Tie
        else:
            return 'C' # Continue
    

## Models

### There are three player models that we want to focus on:

1. Random model
  - In this model, the decision making is 50/50.
  - This is the most basic model where the chance of the player choosing hitting and standing is equal.


2. NEAT model
   - For this model, we will be using a genetic algorithm.
   - The player will keep an array of card values and see what its parents for the values given.
   - They will determine whether or not if they should hit or stayed based on the card and their history with it.
   
   
3. Reinforcement learning model
   - This model will be using a reward based algorithm, or reinforcement learning.
   - The player will receive a reward for winning and a 'punishment' for losing.


### First, we have to create the "Brain" for each model that handles the fitness, showing graphs, etc.

In [39]:
class Brain():
    
    # The class in charge of setting a population and printing out fitness and the players
    
    def __init__(self):
        self._population = []
    
    def getPopulation(self):
        return self._population
    
    def printFitness(self):
        average = 0
        for x in self._population:
            average += x.returnStats();
        print("Population size:", len(self._population))
        print("Average fitness: {}%".format(round(average/len(self._population)* 100,2)))
        highest_fitness = max(self._population, key=lambda player: player.returnStats())
        print("Highest fitness: {}%".format(round(highest_fitness.returnStats()*100,2)))
        
        
    def printPlayers(self):
        self._population.sort(key= lambda player: player.returnStats())
        print(self._population)
        average = 0
        

#### Random Model

In [34]:
class RandomBrain(Brain):
    
    class RandomModel(Player):

        def __init__(self):
            Player.__init__(self)

        def makeDecision(self, oppositeValue):
            if random.randint(0,1) == 1:
                return True
            return False
        
    def __init__(self):
        Brain.__init__(self)
    
    def addPlayer(self, rand_player):
        assert(isinstance(rand_player, self.RandomModel))
        self._population.append(rand_player)
        
    def startSimulation(self, **kwargs):
        dealer = Dealer()
        
        for i in range(0,kwargs['pop']):
            self.addPlayer(self.RandomModel())
        
        for player in self._population:
                game = Game(player, dealer)
                game.SimulateGames(n=kwargs['games'])
            
        Brain.printFitness(self)
                

#### NEAT Model

In [35]:
class NeatBrain():
    
    class NeatModel(Player):
    
        def __init__(self):
            Player.__init__(self)
            self.__cardsAndAction = {}

        
        def makeDecision(self, oppositeValue):
            value = Player.getValue(self)
            if value not in self.__cardsAndAction:
                self.__cardsAndAction[value] = random.randint(0,1)
            else:
                return self.__cardsAndAction[value]
            
        def passGenes(self, p1, p2):
            for i in range(1,21):
                try:
                    self.__cardsAndAction[str(i)] = random.choice([p1,p2]).__cardsAndAction[str(i)]
                except:
                    self.__cardsAndAction[str(i)] = random.choice([True, False])
                
                if random.random() < .09:
                    self.__cardsAndAction[str(i)] = !(random.choice([p1,p2]).__cardsAndAction[str(i)])
            
    
    def __init__(self):
        Brain.__init__(self)
        
    def addPlayer(self, neat):
        assert(isinstance(neat, self.NeatModel))
        self._population.append(neat)

    def selection(self):
        self._population.sort(key= lambda player: player.returnStats())
        self._population = self._population[len(self._population)-ceil(.3*len(self._population)):]
        return self._population
    
    def createGeneration(self, n):
        while(len(self._population) < n):
            offspring = self.NeatModel()
            p1,p2 = random.choice(self._population), random.choice(self._population)
            while(p1 == p2):
                p2 = random.choice(self._population)
            offspring.passGenes(p1,p2)
            self._population.append(offspring)

    def startSimulation(self, **kwargs):
        dealer = Dealer()
        
        for i in range(0,kwargs['pop']):
            self.addPlayer(self.NeatModel())
        
        for i in range(1, kwargs['generations']+1):
            for player in self._population:
                game = Game(player, dealer)
                game.SimulateGames(n=kwargs['games'])
            self.selection()
            self.createGeneration(kwargs['pop'])
        
        for player in self._population:
                game = Game(player, dealer)
                game.SimulateGames(n=kwargs['games'])
                
        Brain.printFitness(self)

## Testing the models

#### Random Model

In [41]:
random_brain = RandomBrain()
random_brain.startSimulation(pop=500, games=500)

Population size: 500
Average fitness: 26.06%
Highest fitness: 31.8%


#### NEAT Model

In [40]:
neat_brain = NeatBrain()
neat_brain.startSimulation(pop=50, games=50, generations=25)

Population size: 50
Average fitness: 30.52%
Highest fitness: 46.0%
